In [95]:
import requests
import bs4
import pandas as pd
import numpy as np

In [5]:
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [150]:
res = requests.get(url.format(2015))
soup = bs4.BeautifulSoup(res.text,'lxml')
table = soup.find('table')
final_df = pd.read_html(str(table))[0]
    
for col in ['Age','G','FGA','FG','3P','FT','FTA','TRB','AST','STL','BLK','TOV','PTS']:
    final_df[col] = pd.to_numeric(final_df[col], errors='coerce')
    
final_df['2015_score'] = (final_df['FGA']*(-0.45) + final_df['FG']*(1.0) + final_df['FTA']*(-0.75) + final_df['FT']*(1.0) 
                        + final_df['3P']*(3.0) + final_df['PTS']*(0.5) + final_df['TRB']*(1.5) + final_df['AST']*(1.5)
                        + final_df['STL']*(2.0) + final_df['BLK']*(3.0) + final_df['TOV']*(-2.0) + final_df['G']*(0.2) 
                        + final_df['Age']*(1))
                     
final_df = final_df.drop(columns=['Rk', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                          '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                          'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'], axis=1)
                     
final_df.set_index('Player')

for i in range(2016,2021):
    
    res = requests.get(url.format(i))
    soup = bs4.BeautifulSoup(res.text,'lxml')
    table = soup.find('table')
    df = pd.read_html(str(table))[0]
    
    for col in ['Age','G','FGA','FG','3P','FT','FTA','TRB','AST','STL','BLK','TOV','PTS']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df[f'{i}_score'] = (df['FGA']*(-0.45) + df['FG']*(1.0) + df['FTA']*(-0.75) + df['FT']*(1.0) 
                        + df['3P']*(3.0) + df['PTS']*(0.5) + df['TRB']*(1.5) + df['AST']*(1.5)
                        + df['STL']*(2.0) + df['BLK']*(3.0) + df['TOV']*(-2.0) + df['G']*(0.2) 
                        + df['Age']*(1))
                     
    df = df.drop(columns=['Rk', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                          '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                          'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'], axis=1)
                     
    df.set_index('Player')
    
    final_df = pd.merge(final_df,df,on='Player',how='outer')
    
#columns before drop:
#Index['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       #'3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       #'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      #dtype='object'
# useful cat: 'Age','G','FGA','FG','3P','FT','FTA','TRB','AST','STL','BLK','TOV','PTS'
# Age factor is not apply to the score format right now, but will be considered in the future

In [151]:
final_df = final_df.groupby('Player').mean()

In [152]:
final_df.columns

Index(['2015_score', '2016_score', '2017_score', '2018_score', '2019_score',
       '2020_score'],
      dtype='object')

Feature Engineering

In [153]:
# remove players that don't have stats during 2018-2020
final_df = final_df.drop(final_df[final_df['2018_score'].isna() & final_df['2019_score'].isna() 
                                  & final_df['2020_score'].isna()].index)

#fill na with average scoring
final_df = final_df.fillna(final_df.mean(axis=0))


In [154]:
final_df.sort_values(by='2020_score',ascending=False).head(15)

,2015_score,2016_score,2017_score,2018_score,2019_score,2020_score
Player,,,,,,
James Harden,80.655000,82.985000,90.220000,87.98000,95.525,93.615
LeBron James,79.450000,82.605000,88.160000,93.84000,87.645,91.245
Damian Lillard,72.455000,74.835000,77.815000,79.82000,82.110,86.720
Giannis Antetokounmpo,57.755000,66.610000,76.910000,78.26000,83.640,83.785
Hassan Whiteside,62.830000,75.540000,77.580000,68.33500,72.120,80.875
Anthony Davis,76.180000,71.930000,80.365000,83.87500,83.690,80.460
Nikola Vučević,68.490000,66.680000,70.860000,70.93500,83.995,79.710
Kyle Lowry,72.270000,79.630000,79.240000,81.13000,78.070,79.515
Chris Paul,83.090000,81.155000,79.920000,80.69000,77.895,79.435


In [155]:
final_df.sort_values(by='2019_score',ascending=False).head(15)

,2015_score,2016_score,2017_score,2018_score,2019_score,2020_score
Player,,,,,,
James Harden,80.655000,82.985000,90.220,87.980,95.525,93.615000
Russell Westbrook,77.450000,82.705000,93.150,86.680,88.810,77.300000
Paul George,38.610000,75.770000,75.050,76.700,87.850,72.840000
Stephen Curry,82.490000,91.385000,84.515,81.470,87.820,64.620000
LeBron James,79.450000,82.605000,88.160,93.840,87.645,91.245000
Nikola Vučević,68.490000,66.680000,70.860,70.935,83.995,79.710000
Anthony Davis,76.180000,71.930000,80.365,83.875,83.690,80.460000
Giannis Antetokounmpo,57.755000,66.610000,76.910,78.260,83.640,83.785000
Kevin Durant,68.140000,83.785000,83.125,84.475,83.410,49.475935


Load Predict Score from computer

In [74]:
predict_df = pd.read_csv('predict_table.csv',index_col='Player')

In [75]:
predict_df.head()

,GP,FG,FGA,FT,FTA,3PM,PTS,TREB,AST,STL,BLK,TO,Predict_Score
Player,,,,,,,,,,,,,
James Harden,68,9.5,21.2,10.2,11.8,4.6,33.8,6.5,7.5,1.8,0.9,4.5,81.610
Anthony Davis,65,9.0,17.9,7.3,8.6,1.3,26.6,9.4,3.2,1.5,2.3,2.5,68.695
Karl-Anthony Towns,69,8.4,16.8,5.0,6.2,3.3,25.1,10.6,3.7,1.1,1.5,3.4,68.090
Stephen Curry,65,8.0,17.5,6.2,6.8,4.1,26.2,5.4,6.4,1.2,0.5,3.8,63.825
Damian Lillard,67,9.0,19.6,6.8,7.6,4.0,28.7,4.2,7.7,1.1,0.3,3.0,67.380


In [156]:
completed_df = pd.merge(final_df,predict_df,on='Player',how='outer')
completed_df = completed_df.drop(columns=['GP', 'FG', 'FGA', 'FT', 'FTA', '3PM', 'PTS', 'TREB',
                                          'AST', 'STL', 'BLK', 'TO'], axis=1)
completed_df.dropna(inplace=True)
completed_df['Predict_Score'] = pd.to_numeric(completed_df['Predict_Score'], errors='coerce')

In [157]:
completed_df.head()

,2015_score,2016_score,2017_score,2018_score,2019_score,2020_score,Predict_Score
Player,,,,,,,
Aaron Gordon,38.995000,55.945000,57.765000,64.07000,67.32000,64.020000,48.305
Aaron Holiday,54.042135,54.881162,53.741132,49.89386,42.06000,52.950000,34.715
Al Horford,70.860000,76.390000,74.690000,75.20000,75.73000,74.980000,43.075
Alec Burks,46.955000,45.965000,43.170000,50.50000,46.93875,59.656667,33.475
Alex Caruso,54.042135,54.881162,53.741132,39.00500,45.14500,49.400000,29.170


In [158]:
completed_df.sort_values(by='Predict_Score',ascending=False).head(15)

,2015_score,2016_score,2017_score,2018_score,2019_score,2020_score,Predict_Score
Player,,,,,,,
James Harden,80.655000,82.985000,90.220000,87.98000,95.525,93.615,81.610
Giannis Antetokounmpo,57.755000,66.610000,76.910000,78.26000,83.640,83.785,76.140
Anthony Davis,76.180000,71.930000,80.365000,83.87500,83.690,80.460,68.695
Karl-Anthony Towns,54.042135,69.005000,78.250000,79.49000,81.755,77.365,68.090
Damian Lillard,72.455000,74.835000,77.815000,79.82000,82.110,86.720,67.380
Trae Young,54.042135,54.881162,53.741132,49.89386,63.850,72.065,65.945
Joel Embiid,54.042135,54.881162,59.015000,71.24000,81.510,72.360,64.440
Stephen Curry,82.490000,91.385000,84.515000,81.47000,87.820,64.620,63.825
LeBron James,79.450000,82.605000,88.160000,93.84000,87.645,91.245,63.055


In [159]:
completed_df.to_csv('final_data.csv')

In [160]:
completed_df.columns

Index(['2015_score', '2016_score', '2017_score', '2018_score', '2019_score',
       '2020_score', 'Predict_Score'],
      dtype='object')

In [161]:
completed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, Aaron Gordon to Zion Williamson
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   2015_score     240 non-null    float64
 1   2016_score     240 non-null    float64
 2   2017_score     240 non-null    float64
 3   2018_score     240 non-null    float64
 4   2019_score     240 non-null    float64
 5   2020_score     240 non-null    float64
 6   Predict_Score  240 non-null    float64
dtypes: float64(7)
memory usage: 15.0+ KB


Training Model

In [168]:
from sklearn.model_selection import train_test_split

X = completed_df[['2015_score', '2016_score', '2017_score', '2018_score', '2019_score','2020_score']] 

y = completed_df['Predict_Score'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [171]:
#Linera Regression Model

from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train,y_train) 
lm_predictions = lm.predict(X_test)

from sklearn import metrics

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, lm_predictions)))

RMSE: 7.423105650439219


In [174]:
#Random Forests

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, rf_pred)))


RMSE: 7.329156453614735


In [185]:
#Neural Networks

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout

model = Sequential()

model.add(Dense(8,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='relu'))

model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
model.fit(X_train,y_train,epochs=800,validation_data=(X_test, y_test),callbacks=[early_stop])

training_score = model.evaluate(X_train,y_train,verbose=0)
test_score = model.evaluate(X_test,y_test,verbose=0)

test_predictions = model.predict(X_test)

print('training_score: ', training_score)
print('test_score: ', test_score)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, test_predictions)))

Epoch 1/800

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

6/6 [==============================] - 0s 18ms/step - loss: 1406.8003 - val_loss: 1675.2396
Epoch 2/800
6/6 [==============================] - 0s 4ms/step - loss: 1368.1833 - val_loss: 1674.7714
Epoch 3/800
6/6 [==============================] - 0s 4ms/step - loss: 1398.3584 - val_loss: 1674.3102
Epoch 4/800
6/6 [==============================] - 0s 4ms/step - loss: 1321.8905 - val_loss: 1673.8604
Epoch 5/800
6/6 [==============================] - 0s 4ms/step - loss: 1368.8788 - val_loss: 16

6/6 [==============================] - 0s 4ms/step - loss: 966.9243 - val_loss: 686.7019
Epoch 74/800
6/6 [==============================] - 0s 4ms/step - loss: 865.4178 - val_loss: 689.0740
Epoch 75/800
6/6 [==============================] - 0s 4ms/step - loss: 962.7518 - val_loss: 677.7546
Epoch 76/800
6/6 [==============================] - 0s 4ms/step - loss: 983.1506 - val_loss: 672.7083
Epoch 77/800
6/6 [==============================] - 0s 4ms/step - loss: 870.7650 - val_loss: 680.9711
Epoch 78/800
6/6 [==============================] - 0s 4ms/step - loss: 968.0342 - val_loss: 679.9722
Epoch 79/800
6/6 [==============================] - 0s 4ms/step - loss: 916.1729 - val_loss: 671.5338
Epoch 80/800
6/6 [==============================] - 0s 4ms/step - loss: 932.4128 - val_loss: 663.1245
Epoch 81/800
6/6 [==============================] - 0s 4ms/step - loss: 790.1495 - val_loss: 680.8430
Epoch 82/800
6/6 [==============================] - 0s 4ms/step - loss: 806.1555 - val_loss: 69